In [2]:
# Necessary imports
import gym
from gym import spaces
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import base64, io
import numpy as np
from collections import deque, namedtuple

# For visualization
from IPython.display import HTML
from IPython import display 
import glob

In [ ]:
test = {}
for i in range(2):
    test['req_{}'.format(i)] = np.full((288, 6),0)
test

In [ ]:
test = generate_scenario(15000,2,generate_deterministic_capacity_dict(2, [6,6]))

In [38]:
#Function for normal distribution truncation:

from scipy.stats import truncnorm

def get_truncated_normal(mean, sd, low, upp):
    #convert timeslot
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

#Function to get the one-hot-encoded vectors for departure and arrival airports:

def one_hot_encode_airport(airport, num_airports):
    encoding = np.zeros(num_airports)
    encoding[airport] = 1
    return encoding

# Example usage
num_airports = 2
airport1 = 0
airport2 = 1

encoded_airport1 = one_hot_encode_airport(airport1, num_airports)
encoded_airport2 = one_hot_encode_airport(airport2, num_airports)

print(encoded_airport1)
print(encoded_airport2)

#Generate full info for the arrival sides:

def generate_info_arv(requests):
    ts_arv = np.empty(shape=(len(requests),), dtype='object')
    start_date_arv = np.empty(shape=(len(requests),), dtype='object')
    for i in range(len(requests)):
        ts_arv[i] = requests[i][1] + requests[i][5]
        # this need to change to 24 hrs
        if ts_arv[i] >= 2400:
            ts_arv[i] = ts_arv[i] - 2400
            start_date_arv[i] = requests[i][2] + 1
        else:
            start_date_arv[i] = requests[i][2]
    return ts_arv, start_date_arv

#for benchmark 2 
def convert_to_timeslot(ts):
    return (ts//12 *100) + (((ts//12) * 60 + (ts % 12)* 5) % 60)

def convert_from_timeslot(ts):
    return (ts//100) * 12 + (ts % 100) // 5


#Modify the distribution based on historical data later:
def generate_scenario(number_of_requests, num_airports, cap_dict):
    
    #number_of_requests = 15000
    ts_72 = get_truncated_normal(mean=72, sd=12, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_72 = np.round(ts_72)

    ts_216 = get_truncated_normal(mean=216, sd=12, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_216 = np.round(ts_216)

    ts_dep = np.concatenate((ts_72, ts_216))
    ts_dep = ts_dep.astype(int)
    ##Edited for benchmark 2 -- converting timeslot to time
    for i, ts_label in enumerate(ts_dep):
        ts_dep[i] = convert_to_timeslot(ts_dep[i])
  
    #Generate start date:

    start_date_dep = np.random.randint(low = 0, high=146, size=number_of_requests) #146 because period is 182 days and we consider series which span at least 5 weeks (+35 days)

    #Generate index for requests:

    index = np.array(list(range(number_of_requests)))

    #Generate origin (0 and 1 are two considered origin airports, 2 represent other airports, encoded in one-hot vector):

    #num_airports = 3
    origin_airport = np.empty(shape=(number_of_requests,), dtype='object')
    destination_airport = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range(number_of_requests):
        _org_airport = one_hot_encode_airport(random.randint(0,1), num_airports)
        _org_airport_list = _org_airport.tolist()
        origin_airport[i] = _org_airport_list
        #Generate destination (the destination will be different with the origin):
        _dest_airport = _org_airport.copy()
        while np.array_equal(_dest_airport, _org_airport):
            np.random.shuffle(_dest_airport)
        _dest_airport_list = _dest_airport.tolist()
        destination_airport[i] = _dest_airport_list

    #Generate flying time (assume between airport 0 and 1 is 2 hour, 0 to 2 and 1 to 2 is arbitrary):

    fly_time = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range (number_of_requests):
        if origin_airport[i] == list([1.0, 0.0]) and destination_airport[i] == list([0.0, 1.0]):
            #Edited for benchmark 2: change flytime to time conversion
            fly_time[i] = 200
        elif origin_airport[i] == list([0.0, 1.0]) and destination_airport[i] == list([1.0, 0.0]):
            fly_time[i] = random.choice([100, 200, 300])

    #Generate status cap:

    status_cap_dep = np.full((number_of_requests,), 0)
    status_cap_arv = np.full((number_of_requests,), 0)
    

    requests = np.stack((index, ts_dep, start_date_dep, origin_airport, destination_airport, fly_time, status_cap_dep), axis=1)

    #Generate full info for the arv side:
    ts_arv, start_date_arv = generate_info_arv(requests)
    #pseudo_belong_dep = np.full((number_of_requests,), 0)
    #pseudo_belong_arv = np.full((number_of_requests,), 0)
    
    # Define requests_full as dtype object
    # requests_full = np.stack((index, ts_dep, start_date_dep, num_of_weeks, date_seq_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, start_date_arv, date_seq_arv, status_cap_arv), axis=1)
    num_entries = len(index)  # Given that 'index' is defined using np.array(list(range(number_of_requests)))
    # Create an empty array of the desired shape with dtype=object
    requests_full = np.empty((num_entries, 10), dtype=object)
    # Fill the array
    data = [index, ts_dep, start_date_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, start_date_arv, status_cap_arv]
    for i, column_data in enumerate(data):
        requests_full[:, i] = column_data

    # airport_req_dict: A dictionary where each key corresponds to a specific airport's requirements. 
    # The key format is 'req_i', where i is the index of the airport. 
    # The value for each key is a numpy array, with each row representing a request and the columns containing different attributes of that request.
    airport_req_dict, _belong_airport_dict = get_airport_req_dict(requests_full, num_airports)

    pot_dem_dict = get_initial_pot_dem_per_airport(airport_req_dict, num_airports)
        
    cap_dem_dict = get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict)
    # for i, label in enumerate(requests_full):
    #     print(requests_full[i])
    return requests_full, airport_req_dict, _belong_airport_dict, pot_dem_dict, cap_dem_dict

# def flatten_cap_dem_dict(cap_dem_dict, num_airports):
#     cap_dem_dict_flat = {}
#     for i in range(num_airports):
#         cap_dem_dict_flat['req_{}'.format(i)] = cap_dem_dict['req_{}'.format(i)].flatten()
#     return cap_dem_dict_flat

#Get separated req per airport and store in a dict:

def get_airport_req_dict(requests_full, num_airports):
    airport_req_dict = {}
    _belong_airport_dict = {}
    for i in range(num_airports):
        airport_req_dict['req_{}'.format(i)] = np.empty((0, 12)) #This one depends on the number of elements of a final request
        _belong_airport_dict['req_{}'.format(i)] = np.full(num_airports, 0.0, dtype=float)
        _belong_airport_dict['req_{}'.format(i)][i] = float(1.0)
        _belong_airport_dict['req_{}'.format(i)] = _belong_airport_dict['req_{}'.format(i)].tolist()
        
    for i in range(len(requests_full)):
        _found_dep = 0
        _found_arv = 0
        for k in range(num_airports):
            #_found_dep = 0
            #_found_arv = 0
            if requests_full[i][3] == _belong_airport_dict['req_{}'.format(k)]:
                _dep_req = np.append(requests_full[i], 1)
                _dep_req = np.append(_dep_req, 0)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _dep_req))
                _found_dep = 1
                #airport_req_dict['req_{}'.format(k)] = np.append(airport_req_dict['req_{}'.format(k)], 1)
                #airport_req_dict['req_{}'.format(k)] = np.append(airport_req_dict['req_{}'.format(k)], 0)
                #break
            if requests_full[i][4] == _belong_airport_dict['req_{}'.format(k)]:
                _arv_req = np.append(requests_full[i], 0)
                _arv_req = np.append(_arv_req, 1)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _arv_req))
                _found_arv = 1
                #airport_req_dict['req_{}'.format(k)] = np.append(airport_req_dict['req_{}'.format(k)], 0)
                #airport_req_dict['req_{}'.format(k)] = np.append(airport_req_dict['req_{}'.format(k)], 1)
                #break
            if _found_dep + _found_arv == 2:
                break
        if _found_dep + _found_arv != 2:
            print("Cannot found both dep and arv at req {}".format(i))
    
    return airport_req_dict, _belong_airport_dict


##Edited for benchmark 2 -- changing to 288 x 6 array 
def generate_deterministic_capacity_dict(num_airports, cap_per_airport_arr):
    cap_dict = {}
    for i in range(num_airports):
        cap_dict['req_{}'.format(i)] = np.full((288, cap_per_airport_arr[i]), 0)
    return cap_dict

def get_initial_pot_dem_per_airport(airport_req_dict, num_airports): #Replace req_df to req_df_update to update pot_dem_df #To be replaced with final_sched
    pot_dem_dict = {}
    #TODO: increase speed
    #10 dep 11 arv, 1 dep ts, 7 arv ts
    for i in range(num_airports):
        pot_dem_dict['req_{}'.format(i)] = np.full((288), 0)
        for k in range(len(airport_req_dict['req_{}'.format(i)])):
            _time_slot = int(airport_req_dict['req_{}'.format(i)][k][1]) * int(airport_req_dict['req_{}'.format(i)][k][10]) + int(airport_req_dict['req_{}'.format(i)][k][7]) * int(airport_req_dict['req_{}'.format(i)][k][11])
            _time_index = convert_from_timeslot(_time_slot)
            pot_dem_dict['req_{}'.format(i)][_time_index] += 1
    return pot_dem_dict

def get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict):
    cap_dem_dict = {}
    cap_size_dict = {}
    for i in range(num_airports):
        cap_size_dict['req_{}'.format(i)] = np.full((288), cap_dict['req_{}'.format(i)].shape[1])
        cap_dem_dict['req_{}'.format(i)] = cap_dict['req_{}'.format(i)] - pot_dem_dict['req_{}'.format(i)]
    return cap_dem_dict

def get_violate_id_set(airport_req_dict, num_airports):
    violate_set = [] #(1: id, 2: airport, 3: dep, 4: arv)
    for i in range(num_airports):
        mask = ((airport_req_dict['req_{}'.format(i)][:, 6] + airport_req_dict['req_{}'.format(i)][:, 9]) >= 1)
        _id_violate_per_airport = airport_req_dict['req_{}'.format(i)][mask, :][:,0]
        violate_set.append(_id_violate_per_airport)
    violate_set = np.concatenate(violate_set, axis=0)
    violate_set = np.unique(violate_set)
    return violate_set

def get_violate_id_set_req_full(requests_full):
    mask = ((requests_full[:, 8] + requests_full[:, 12]) >= 1)
    violate_set_req_full = requests_full[mask, :][:,0]
    return violate_set_req_full

def get_req(violate_set, requests_full):
    _violate_index = random.choice(violate_set)
    chosen_req = requests_full[requests_full[:,0] == _violate_index]
    return chosen_req
#still need update status cap
# def update_status_capacity(airport_req_dict, num_airports, request_full):
#     for i in range(num_airports):


[1. 0.]
[0. 1.]


In [ ]:
class TwoAirportSchedEnv(gym.Env):
    def __init__(self, number_of_actions, number_of_requests, num_airports, cap_per_airport_arr):
        super(TwoAirportSchedEnv, self).__init__()
        self.number_of_actions = int(number_of_actions)
        self.number_of_requests = number_of_requests
        self.num_airports = num_airports
        self.cap_per_airport_arr = cap_per_airport_arr
        self.cap_dict = generate_deterministic_capacity_dict(self.num_airports, self.cap_per_airport_arr)
        #Benchmark 2
        _exceed_cap = 0         
        while _exceed_cap == 0:
            self.requests_full, self.airport_req_dict, self._belong_airport_dict = generate_scenario(number_of_requests = self.number_of_requests, num_airports = self.num_airports, cap_dict = self.cap_dict)
            self.cap_dem_dict_flat = flatten_cap_dem_dict(self.cap_dem_dict, self.num_airports)
            for i in range(num_airports):
                _exceed_cap = _exceed_cap + (min(self.cap_dem_dict_flat['req_{}'.format(i)]))

        #to generate action:
        self.action_space = spaces.Discrete(self.number_of_actions)
        self.observation_space = spaces.Box(low= -np.inf, high= np.inf, shape=(7,), dtype=float)
        self.violate_set = self.get_violate_id_set(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
        self.num_step = 0
        
        #to choose the request that will be checked
        self.get_req(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        self.dep_time_slot = self.chosen_req[1]
        self.original_requests = self.requests_full

        self.agent1 = Agent(state_size=578, action_size=self.number_of_actions, seed=0)
        self.agent2 = Agent(state_size=578, action_size=self.number_of_actions, seed=0)

        self.agent_dict = {}
        self.agent_dict[0] = self.agent1
        self.agent_dict[1] = self.agent2

    def get_agent(self, airport_type):
        # Check for airport type
        if airport_type == "departure":
            airport_index = self.chosen_req[3].index(1.0)
        elif airport_type == "arrival":
            airport_index = self.chosen_req[4].index(1.0)
        else:
            raise ValueError(f"Invalid airport_type: {airport_type}. Expected 'departure' or 'arrival'.")

        agent = self.agent_dict.get(airport_index)
        return agent
    
    def action_mapping(self, action, number_of_actions):
        mid_point = (number_of_actions - 1) // 2
        return action - mid_point
    
    #added get request to know which is the request we are going to move at this step",
    #Randomly choose a violating request to solve
    def get_req(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):

        if not self.violate_set:
          self.violate_set = self.get_violate_id_set(airport_req_dict, num_airports, cap_dem_dict, requests_full)
          if not self.violate_set:
            self.chosen_req = None 
            return self.chosen_req 

        airport_num, airport_index, request_violate_index = random.choice(self.violate_set)
        valid = self.check_status_capacity(airport_req_dict, cap_dem_dict, requests_full, airport_num, airport_index, request_violate_index)

        while not valid:

          if not self.violate_set:
            self.violate_set = self.get_violate_id_set(airport_req_dict, num_airports, cap_dem_dict, requests_full)
            if not self.violate_set:
              self.chosen_req = None 
              return self.chosen_req 

          airport_num, airport_index, request_violate_index = random.choice(self.violate_set)
          valid = self.check_status_capacity(airport_req_dict, cap_dem_dict, requests_full, airport_num, airport_index, request_violate_index)

        self.chosen_req = self.requests_full[self.requests_full[:,0] == request_violate_index][0]

        return self.chosen_req
    
    #need to check arrival time or change the variable to check_outbound(arv) and chec_outbound(dep)
    def check_outbound(self, action):
        dep_time_slot = self.chosen_req[1]
        arv_time_slot = self.chosen_req[7]

        outbound = False

        new_dep_time_slot = dep_time_slot + action
        new_arv_time_slot = arv_time_slot + action

        if (new_dep_time_slot < 0) or (new_dep_time_slot > 287) or (new_arv_time_slot < 0) or (new_arv_time_slot > 287):
            outbound = True

        return outbound
    
    def dep_or_arv(self):
        pass

    def update_dem(self, dep_airport, arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv):
        # Increment demand for the new time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][new_time_slot_dep] += 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][new_time_slot_arv] += 1

        # Decrease demand for the initial time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][time_slot_dep] -= 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][time_slot_arv] -= 1

    def update_cap_dem(self):
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            if cap_key not in self.cap_dict:
                raise KeyError(f"'{cap_key}' not found in cap_dict. Available keys: {list(self.cap_dict.keys())}")
            self.cap_dem_dict[cap_key] = self.cap_dict[cap_key] - self.pot_dem_dict[dem_key]

    def check_status_capacity(self, airport_req_dict, cap_dem_dict, requests_full, airport_num, airport_index, request_index):

        _time_slot = int(airport_req_dict['req_{}'.format(airport_num)][airport_index][1]) * int(airport_req_dict['req_{}'.format(airport_num)][airport_index][10]) + int(airport_req_dict['req_{}'.format(airport_num)][airport_index][7]) * int(airport_req_dict['req_{}'.format(airport_num)][airport_index][11])

        if (airport_num, airport_index, request_index) in self.violate_set:
            self.violate_set.remove((airport_num, airport_index, request_index))

        cap_dem_values = cap_dem_dict['req_{}'.format(airport_num)][_time_slot]
        if isinstance(cap_dem_values, (np.ndarray, list)) and all(x >= 0 for x in cap_dem_values):
            airport_req_dict['req_{}'.format(airport_num)][airport_index][6] = 0
            airport_req_dict['req_{}'.format(airport_num)][airport_index][9] = 0
            requests_full[request_index, 6] = 0
            requests_full[request_index, 9] = 0

            valid = False
        else:
            if airport_req_dict['req_{}'.format(airport_num)][airport_index][10] == 1:
                airport_req_dict['req_{}'.format(airport_num)][airport_index][6] = 1
                requests_full[request_index, 6] = 1
            else:
                airport_req_dict['req_{}'.format(airport_num)][airport_index][9] = 1
                requests_full[request_index, 9] = 1

            valid = True

        return valid
    
    def update_status_capacity(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):
    #Them cot cap_status o init:
        not_violate_update = set()
        violate_update = set()
        for i in range(num_airports):
            for k in range(len(airport_req_dict['req_{}'.format(i)])):
                _time_slot = int(airport_req_dict['req_{}'.format(i)][k][1]) * int(airport_req_dict['req_{}'.format(i)][k][10]) + int(airport_req_dict['req_{}'.format(i)][k][7]) * int(airport_req_dict['req_{}'.format(i)][k][11])
                cap_dem_values = cap_dem_dict['req_{}'.format(i)][_time_slot]
                if isinstance(cap_dem_values, (np.ndarray, list)) and all(x >= 0 for x in cap_dem_values):
                # if all(x >= 0 for x in cap_dem_dict['req_{}'.format(i)][_time_slot]):
                    if airport_req_dict['req_{}'.format(i)][k][6] == 1 or airport_req_dict['req_{}'.format(i)][k][9] == 1:
                        not_violate_update.add(airport_req_dict['req_{}'.format(i)][k][0])
                    airport_req_dict['req_{}'.format(i)][k][6] = 0
                    airport_req_dict['req_{}'.format(i)][k][9] = 0
                else:
                    if airport_req_dict['req_{}'.format(i)][k][6] == 0 and airport_req_dict['req_{}'.format(i)][k][9] == 0:
                        violate_update.add(airport_req_dict['req_{}'.format(i)][k][0])
                    if airport_req_dict['req_{}'.format(i)][k][10] == 1:
                        airport_req_dict['req_{}'.format(i)][k][6] = 1
                        _indices = np.where(requests_full[:, 0] == airport_req_dict['req_{}'.format(i)][k][0])
                        requests_full[_indices, 6] = 1
                    else:
                        airport_req_dict['req_{}'.format(i)][k][9] = 1
                        _indices = np.where(requests_full[:, 0] == airport_req_dict['req_{}'.format(i)][k][0])
                        requests_full[_indices, 9] = 1

        return not_violate_update, violate_update

    def get_violate_id_set(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):
        self.update_status_capacity(airport_req_dict, num_airports, cap_dem_dict, requests_full)
        violate_set = [] #(1: id, 2: airport, 3: dep, 4: arv)
        for i in range(num_airports):
            for k in range(len(airport_req_dict['req_{}'.format(i)])):
                if airport_req_dict['req_{}'.format(i)][k][6] + airport_req_dict['req_{}'.format(i)][k][9] >= 1:
                    violate_set.append((i, k, airport_req_dict['req_{}'.format(i)][k][0]))

        return violate_set

    # Need to validate
    def update_violate_set(curr_violate, not_violate_update, violate_update):
        curr_violate = set(curr_violate)
        for req_index in not_violate_update:
            curr_violate.remove(req_index)
        for req_index in violate_update:
            curr_violate.add(req_index) 
        curr_violate = list(curr_violate)
        return curr_violate 
    
    # Based on agent's actions
    def take_action(self, action_dep, action_arv):
        #Check if actions match
        if action_dep != action_arv:
            return 0
        else:
            return action_dep
    
    def when_take_action(self, action):   
        _index = self.chosen_req[0]
        time_slot_dep = self.chosen_req[1]
        time_slot_arv = self.chosen_req[7]
        time_slot_arv = int(time_slot_arv)
        _dep_airport = self.chosen_req[3].index(1.0)
        _arv_airport = self.chosen_req[4].index(1.0)
        new_time_slot_dep = time_slot_dep + action
        new_time_slot_arv = time_slot_arv + action
        new_time_slot_arv = int(new_time_slot_arv)

        outbound = self.check_outbound(action)

        if not outbound:
            pass
            
        elif outbound:
            new_time_slot_dep = random.randint(0, 287)
            new_time_slot_arv = random.randint(0, 287)

        else:
            print('Problem with check outbound!')
            
        # Update the request for both departure and arrival time slots and dates
        self.requests_full[self.requests_full[:, 0] == _index][:, 1] = new_time_slot_dep
        self.requests_full[self.requests_full[:, 0] == _index][:, 7] = new_time_slot_arv

        # Update the airport request dict
        self.airport_req_dict['req_{}'.format(_dep_airport)][self.airport_req_dict['req_{}'.format(_dep_airport)][:, 0] == _index][:, 1] = new_time_slot_dep
        self.airport_req_dict['req_{}'.format(_dep_airport)][self.airport_req_dict['req_{}'.format(_dep_airport)][:, 0] == _index][:, 7] = new_time_slot_arv
        self.airport_req_dict['req_{}'.format(_arv_airport)][self.airport_req_dict['req_{}'.format(_arv_airport)][:, 0] == _index][:, 1] = new_time_slot_dep
        self.airport_req_dict['req_{}'.format(_arv_airport)][self.airport_req_dict['req_{}'.format(_arv_airport)][:, 0] == _index][:, 7] = new_time_slot_arv

        # Update demand for the new time slots
        self.update_dem(_dep_airport, _arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv)
        
        # Update cap_dem 
        self.update_cap_dem()

    def step(self, action): 
        num_actions = self.number_of_actions
        # # Convert the discrete action to your desired action value
        real_action = self.action_mapping(action, num_actions)
        
        # observation concats the following:
        # ts_dep + cap_dem_dep + ts_arv + cap_dem_arv
        self.when_take_action(real_action)
        outbound = self.check_outbound(real_action)

        # Reward part:
        done = False
        if real_action != 0:
            
            local_reward = 0
            if outbound:
                # local_reward = -1
                local_reward = 0.1*(-abs(real_action)*0.5)
            else:
                local_reward = 0.1*(-abs(real_action)*0.5) #TODO change if increase number of actions
            
            self.num_step += 1
            if self.chosen_req[6] == 0 and self.chosen_req[9] == 0:
                done = True
                obs = np.zeros(578)
                global_reward = 10

            elif self.num_step == self.number_of_requests*5:
                done = True
                obs = np.zeros(578)
                negative_sum = -10
                for value in self.cap_dem_dict.values():
                    # Assuming each value is a numeric value or a numpy array
                    # If it's a numpy array, you can sum all negative values directly using numpy
                    if isinstance(value, np.ndarray):
                        negative_sum += np.sum(value[value < 0])
                    else:
                        # If it's a single numeric value, just check if it's negative
                        if value < 0:
                            negative_sum += value
                global_reward = negative_sum

            else:
                global_reward = 0
                obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
            reward_time_step = -0.5
        

        else:
            reward_time_step = -0.5
            local_reward = 0
            global_reward = 0
            obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        total_reward = float(local_reward + global_reward + reward_time_step)
    
        return obs, total_reward, done, {}
    
    def _next_observation(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):
        self.get_req(airport_req_dict, num_airports, cap_dem_dict, requests_full)
        _ts_dep = self.chosen_req[1]
        _ts_arv = self.chosen_req[7]
        _dep_airport = self.chosen_req[3].index(1.0)
        _arv_airport = self.chosen_req[4].index(1.0)
        cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]

        self.obs = np.append(_ts_dep, cap_dem_arr_dep)
        self.obs = np.append(self.obs, _ts_arv)
        self.obs = np.append(self.obs, cap_dem_arr_arv)
        
        return self.obs

    def reset(self):                
        generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
        _dep_airport = self.chosen_req[3].index(1.0)
        _arv_airport = self.chosen_req[4].index(1.0)
        cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]
        
        _cap_dem_flat_dep = cap_dem_arr_dep.flatten()
        _cap_dem_flat_arv = cap_dem_arr_arv.flatten()

        while (min(_cap_dem_flat_dep) >= 0 or min(_cap_dem_flat_arv) >= 0):
            generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
            _cap_dem_flat_dep = cap_dem_arr_dep.flatten()
            _cap_dem_flat_arv = cap_dem_arr_arv.flatten()
        
        update_status_capacity(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
        print('Number of violation for dep: ', len(_cap_dem_flat_dep[_cap_dem_flat_dep < 0]))
        print('Number of violation for arv: ', len(_cap_dem_flat_arv[_cap_dem_flat_arv < 0]))

        self.num_step = 0        
        return self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
    
    # Need to validate
    def eval(self):
        _dep_airport = self.chosen_req[5].index(1.0)
        _arv_airport = self.chosen_req[6].index(1.0)
        _initial_cap_dem_dict = {}
        _initial_pot_dem_dict = get_initial_pot_dem_per_airport(self.airport_req_dict, self.num_airports)
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            _initial_cap_dem_dict[cap_key] = self.cap_dict[cap_key] - _initial_pot_dem_dict[dem_key]

        _initial_cap_dem_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        _initial_cap_dem_flat_dep = _initial_cap_dem_dep.flatten()
        _initial_violate_dep = len(_initial_cap_dem_flat_dep[_initial_cap_dem_flat_dep < 0])
        print('Initial violation of departure is: ', _initial_violate_dep)
        _initial_cap_dem_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]
        _initial_cap_dem_flat_arv = _initial_cap_dem_arv.flatten()
        _initial_violate_arv = len(_initial_cap_dem_flat_arv[_initial_cap_dem_flat_arv < 0])
        print('Initial violation of arrival is: ', _initial_violate_arv)

        cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]
        _final_cap_dem_flat_dep = cap_dem_arr_dep.flatten()
        _final_cap_dem_flat_arv = cap_dem_arr_arv.flatten()
        _final_violate_dep = len(_final_cap_dem_flat_dep[_final_cap_dem_flat_dep < 0])
        _final_violate_arv = len(_final_cap_dem_flat_arv[_final_cap_dem_flat_arv < 0])
        print('Final violation of departure is: ', _final_violate_dep)
        print('Final violation of arrival is: ', _final_violate_arv)

        _total_sched_delay_dep = sum(abs(self.original_requests[:,1] - self.requests_full[:,1])* self.original_requests[:,3])
        print('Total schedule delay of departure is: ', _total_sched_delay_dep)
        _total_sched_delay_arv = sum(abs(self.original_requests[:,9] - self.requests_full[:,9])* self.original_requests[:,3])
        print('Total schedule delay of arrival is: ', _total_sched_delay_arv)

        _max_shift_dep = max(abs(self.original_requests[:,1] - self.requests_full[:,1]))
        print('Max shift of departure: ', _max_shift_dep)
        _max_shift_arv = max(abs(self.original_requests[:,9] - self.requests_full[:,9]))
        print('Max shift of arrival: ', _max_shift_arv)

        _unaccom_req_dep = len(self.requests_full[self.requests_full[:,8] == 1])
        print('Number of unaccommodate departure requests: ', _unaccom_req_dep)
        _unaccom_req_arv = len(self.requests_full[self.requests_full[:,12] == 1])
        print('Number of unaccommodate arrival requests: ', _unaccom_req_arv)

        return _initial_violate_dep, _initial_violate_arv, _final_violate_dep, _final_violate_arv, _total_sched_delay_dep, _total_sched_delay_arv, _max_shift_dep, _max_shift_arv, _unaccom_req_dep, _unaccom_req_arv